In [1]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 9.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 36.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [48]:
import fitz
with fitz.open("PDFs/file_2.pdf") as doc:
    page = doc[0]
    word_list = page.get_text("words",sort=True)
#    print(text)
    # text = text.replace("\n", " ")

In [49]:
for i in word_list:
    print(i)

(265.94000244140625, 19.25, 292.05999755859375, 32.9900016784668, 'Reset', 97, 1, 0)
(322.7200012207031, 19.25, 343.2799987792969, 32.9900016784668, 'Print', 97, 0, 0)
(32.917999267578125, 58.76129150390625, 73.46968078613281, 69.93316650390625, 'HEALTH', 0, 0, 0)
(76.24800109863281, 58.76129150390625, 135.68792724609375, 69.93316650390625, 'INSURANCE', 0, 0, 1)
(138.46800231933594, 58.76129150390625, 170.12808227539062, 69.93316650390625, 'CLAIM', 0, 0, 2)
(172.9080047607422, 58.76129150390625, 202.34808349609375, 69.93316650390625, 'FORM', 0, 0, 3)
(32.917999267578125, 71.223388671875, 66.59100341796875, 77.926513671875, 'APPROVED', 0, 1, 0)
(68.25799560546875, 71.223388671875, 76.59194946289062, 77.926513671875, 'BY', 0, 1, 1)
(78.25999450683594, 71.223388671875, 109.25503540039062, 77.926513671875, 'NATIONAL', 0, 1, 2)
(110.92399597167969, 71.223388671875, 138.9200439453125, 77.926513671875, 'UNIFORM', 0, 1, 3)
(140.58799743652344, 71.223388671875, 159.58404541015625, 77.9265136718

In [24]:
import pandas as pd

In [25]:
gtdf = pd.read_excel("gt_100.xlsx")

In [26]:
gtdf_row = gtdf[gtdf["filename"]=="PDFs/file_2.pdf"]

In [27]:
gtdf_row

PERSON  \
1  [[211, 144, 220, 157, 'J'], [121, 144, 207, 15...   

                                             DISEASE  \
1  [[40, 450, 108, 459, 'Influenza'], [40, 459, 1...   

                                            CHEMICAL  \
1  [[202, 513, 264, 526, 'Farxiga'], [202, 531, 2...   

                                              DOSAGE         filename  
1  [[265, 513, 334, 526, '600 mg'], [265, 531, 33...  PDFs/file_2.pdf

In [60]:
print(word_list[350:], sep = '\n')

[(144.27499389648438, 414.72235107421875, 176.6219024658203, 421.42547607421875, '(Designated', 49, 0, 4), (178.2889862060547, 414.72235107421875, 184.62498474121094, 421.42547607421875, 'by', 49, 0, 5), (186.29298400878906, 414.72235107421875, 205.61903381347656, 421.42547607421875, 'NUCC)', 49, 0, 6), (383.9170227050781, 414.8063659667969, 392.2560119628906, 421.5094909667969, '20.', 51, 0, 0), (393.9250183105469, 414.8063659667969, 420.594970703125, 421.5094909667969, 'OUTSIDE', 51, 0, 1), (422.2630310058594, 414.8063659667969, 436.9399719238281, 421.5094909667969, 'LAB?', 51, 0, 2), (496.4170227050781, 414.8063659667969, 499.7539367675781, 421.5094909667969, '$', 51, 1, 0), (501.4210205078125, 414.8063659667969, 531.0910034179688, 421.5094909667969, 'CHARGES', 51, 1, 1), (396.67401123046875, 426.0563659667969, 407.01202392578125, 432.7594909667969, 'Yes', 50, 0, 0), (441.6910095214844, 426.0563659667969, 449.35992431640625, 432.7594909667969, 'No', 50, 1, 0), (32.917999267578125, 4

In [61]:
def overlap(bbox, gt_bbox):
    x_overlap = min(bbox[2], gt_bbox[2]) - max(bbox[0], gt_bbox[0])
    y_overlap = min(bbox[3], gt_bbox[3]) - max(bbox[1], gt_bbox[1])
    if x_overlap > 0 and y_overlap > 0:
        overlap_area = x_overlap*y_overlap
        bbox_area = (bbox[2]-bbox[0])*(bbox[3]-bbox[1])
        if overlap_area/bbox_area > 0.6:
            return True
    return False       


def get_ent_type(bbox, gtdf_row):
    ent_type = None
    for col in gtdf_row.columns:
        if "filename" not in col:
            for row in gtdf_row[col]:
                for gt_box in eval(row):
                    gt_bbox = gt_box[:4]
                    if overlap(bbox, gt_bbox):
                        ent_type = col
                        return ent_type
    return ent_type
                    

In [65]:
TRAIN_DATA = []

In [83]:
text = ''
entities = {"entities": [] }
for i in range(len(word_list)):
    word = word_list[i][4]
#    print(word)
    bbox = [int(w) for w in word_list[i][:4]]
    word_ent = get_ent_type(bbox, gtdf_row)
    if word_ent:
        start_idx = len(text) + 2
        end_idx = start_idx + len(word)
        text += " " + word
#        print(text)
        entities["entities"].append((start_idx, end_idx, word_ent))
    # else:
    #     text += " " + word
TRAIN_DATA = [(text, entities)]
    
    

 Smith
 Smith Alexander
 Smith Alexander J
 Smith Alexander J Influenza
 Smith Alexander J Influenza Needlestick
 Smith Alexander J Influenza Needlestick and
 Smith Alexander J Influenza Needlestick and Sharps
 Smith Alexander J Influenza Needlestick and Sharps Tuberculosis
 Smith Alexander J Influenza Needlestick and Sharps Tuberculosis Beryllium
 Smith Alexander J Influenza Needlestick and Sharps Tuberculosis Beryllium -
 Smith Alexander J Influenza Needlestick and Sharps Tuberculosis Beryllium - Health
 Smith Alexander J Influenza Needlestick and Sharps Tuberculosis Beryllium - Health Effects
 Smith Alexander J Influenza Needlestick and Sharps Tuberculosis Beryllium - Health Effects Farxiga
 Smith Alexander J Influenza Needlestick and Sharps Tuberculosis Beryllium - Health Effects Farxiga 600
 Smith Alexander J Influenza Needlestick and Sharps Tuberculosis Beryllium - Health Effects Farxiga 600 mg
 Smith Alexander J Influenza Needlestick and Sharps Tuberculosis Beryllium - Health Ef

In [85]:
TRAIN_DATA

[(' Smith Alexander J Influenza Needlestick and Sharps Tuberculosis Beryllium - Health Effects Farxiga 600 mg Entresto 50 mg/mL Benadryl 200 mg/5 mL Augmentin 250 mg',
  {'entities': [(2, 7, 'PERSON'),
    (8, 17, 'PERSON'),
    (18, 19, 'PERSON'),
    (20, 29, 'DISEASE'),
    (30, 41, 'DISEASE'),
    (42, 45, 'DISEASE'),
    (46, 52, 'DISEASE'),
    (53, 65, 'DISEASE'),
    (66, 75, 'DISEASE'),
    (76, 77, 'DISEASE'),
    (78, 84, 'DISEASE'),
    (85, 92, 'DISEASE'),
    (93, 100, 'CHEMICAL'),
    (101, 104, 'DOSAGE'),
    (105, 107, 'DOSAGE'),
    (108, 116, 'CHEMICAL'),
    (117, 119, 'DOSAGE'),
    (120, 125, 'DOSAGE'),
    (126, 134, 'CHEMICAL'),
    (135, 138, 'DOSAGE'),
    (139, 143, 'DOSAGE'),
    (144, 146, 'DOSAGE'),
    (147, 156, 'CHEMICAL'),
    (157, 160, 'DOSAGE'),
    (161, 163, 'DOSAGE')]})]